# **모델 사용하기**

## **1.환경준비**

### (1) 구글 드라이브

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(langchain)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [2]:
# 경로 : /content/drive/MyDrive/langchain/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r /content/drive/MyDrive/langchain/requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

#### 2) 라이브러리 로딩

In [3]:
import pandas as pd
import numpy as np
import os
import openai

from langchain.document_loaders import PyMuPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (3) OpenAI API Key 확인

In [4]:
def load_api_key(filepath):
    with open(filepath, 'r') as file:
        return file.readline().strip()

path = '/content/drive/MyDrive/langchain/'

# API 키 로드 및 환경변수 설정
openai.api_key = load_api_key(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key

* ⚠️ 아래 코드셀은, 실행해서 key가 제대로 보이는지 확인하고 삭제하세요.

In [ ]:
print(os.environ['OPENAI_API_KEY'])

## **2.랭체인 모델 사용하기1**
* 입력

### (1) 랭체인으로 LLM 모델 연결

In [6]:
# langeuage_models
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(model = "gpt-3.5-turbo")

### (2) HumanMessage

In [ ]:
result = chat([HumanMessage(content="안녕하세요!")])
print(result.content)

안녕하세요! 무엇을 도와드릴까요?


### (3) SystemMessage

In [ ]:
sys_role = '당신은 애국심을 가지고 있는 건전한 대한민국 국민입니다.'
question = "독도는 어느나라 땅이야?"

result = chat([HumanMessage(content = question), SystemMessage(content = sys_role)])
print(result.content)

독도는 대한민국의 땅입니다.


### (4) PromptTemplate

* 프롬프트 템플릿 설정

In [7]:
from langchain import PromptTemplate

prompt = PromptTemplate(template = "{nation}의 인구수는?", input_variables = ["nation"])

In [8]:
print(prompt.format(nation = "한국"))
print(prompt.format(nation = "영국"))

한국의 인구수는?
영국의 인구수는?


* 언어 모델과 결합

In [ ]:
result = chat([HumanMessage(content=prompt.format(nation = "한국"))])
print(result.content)

2021년 9월 기준으로 한국의 인구는 약 5천만 명입니다.


### 😀실습

* 다음 제공되는 국가 리스트(nations)를 이용해서
* 해당 국가의 인구수를 숫자로 뽑아 리스트에 담아 봅시다.

In [ ]:
nations = ['한국','중국','일본','인도','미국','영국']

## **3.랭체인 모델 사용하기2**
* 출력

### (1) 랭체인으로 LLM 모델 연결

In [ ]:
# langeuage_models
chat = ChatOpenAI(model = "gpt-3.5-turbo")

### (2) Output Parser

In [9]:
from langchain.output_parsers import CommaSeparatedListOutputParser

In [21]:
# 출력파서 선언
output_parser = CommaSeparatedListOutputParser()

# 사용
result = chat([HumanMessage(content = "트랜스포머 기반 언어모델 3개 알려줘."),
               HumanMessage(content = output_parser.get_format_instructions())])

# 결과 출력1
print(result.content)

# 결과 출력2
output = output_parser.parse(result.content)
print(output)

GPT-3, BERT, T5
['GPT-3', 'BERT', 'T5']


### 😀실습

* 충청도의 대표 관광지 5개를 리스트 형태로 출력해 봅시다.

### (3) Streaming
* 답변을 생성하는 대로 화면에 순차적으로 출력

In [24]:
# 기존 방식대로
chat1 = ChatOpenAI(model="gpt-3.5-turbo")

sys_role = '당신은 세계 최고의 요리사이자, 요리 선생님입니다. 당신은 항상 친절하고 정확하게 요리법을 가르칩니다.'
question = "한국식 후라이드치킨 만드는 방법 알려줘?"

result = chat1([HumanMessage(content=question), SystemMessage(content = sys_role)])
print(result.content)

한국식 후라이드치킨을 만드는 방법은 다음과 같습니다:

재료:
- 닭 튀김을 위한 닭 조각 (허벅지, 날개, 가슴 등)
- 밀가루
- 계란
- 빵가루
- 식용유

양념 재료:
- 간장
- 설탕
- 다진 마늘
- 다진 생강
- 후추

1. 닭 조각을 소금과 후추로 조금씩 조미해주세요.
2. 밀가루, 계란, 빵가루를 각각 별도의 그릇에 담아주세요.
3. 닭 조각을 밀가루 → 계란 → 빵가루 순으로 튀김옷을 입혀주세요.
4. 뜨거운 식용유에 닭 조각을 넣고 굽니다. 두 번 튀김하여 바삭하게 만들어주세요.
5. 냉동치킨의 간장 양념을 만들어주세요. 간장, 설탕, 다진 마늘, 다진 생강, 후추를 섞어주세요.
6. 튀김한 닭을 별도의 팬에 옮겨 담고, 양념을 골고루 버무려주세요.
7. 고루 버무린 후라이드치킨을 소스와 함께 내놓아주세요.

이렇게 하면 맛있는 한국식 후라이드치킨이 완성됩니다. 맛있게 즐겨주세요!


In [22]:
# 스트리밍 방식으로 출력
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

chat2 = ChatOpenAI(model="gpt-3.5-turbo",
                   streaming=True, callbacks=[StreamingStdOutCallbackHandler()])  # 스트리밍 설정

sys_role = '당신은 세계 최고의 요리사이자, 요리 선생님입니다. 당신은 항상 친절하고 정확하게 요리법을 가르칩니다.'
question = "한국식 후라이드치킨 만드는 방법 알려줘?"
result = chat2([HumanMessage(content=question), SystemMessage(content = sys_role)])

한국식 후라이드치킨을 만드는 방법은 다음과 같습니다:

재료:
- 닭 튀김용 500g
- 밀가루 1컵
- 계란 2개
- 빵가루 2컵
- 식용유

양념:
- 간장 3큰술
- 설탕 2큰술
- 다진 마늘 1큰술
- 다진 생강 1작은술
- 후추 약간

1. 닭을 손질하고 씻은 뒤 물기를 제거합니다.
2. 닭을 소금과 후추로 조금씩 고루 뿌려줍니다.
3. 밀가루, 계란, 빵가루 순으로 튀김옷을 입혀줍니다.
4. 식용유를 예열한 후 닭을 넣어 170도에서 노릇하게 튀겨줍니다.
5. 다른 팬에 간장, 설탕, 다진 마늘, 다진 생강, 후추를 넣고 센 불에 달궈 양념을 만듭니다.
6. 튀겨진 닭을 양념에 넣어 고루 버무려줍니다.
7. 소스가 고루 베인 후라이드치킨을 접시에 담아 내놓습니다.

이렇게 하면 집에서도 한국식 후라이드치킨을 쉽게 만들 수 있습니다. 맛있게 즐겨보세요!